In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 "shape_predictor_68_face_landmarks.dat.bz2"

In [ ]:

predictor_path = 'shape_predictor_68_face_landmarks.dat'  # path of data

#initializes dlib’s pre-trained face detector based on a modification to the standard Histogram of Oriented Gradients + Linear SVM method for object detection.
detector = dlib.get_frontal_face_detector() 
#loads the facial landmark predictor using the path
predictor = dlib.shape_predictor(predictor_path)

In [ ]:
# take a bounding predicted by dlib and convert it
def rect_to_bb(rect):
	
	# to the format (x, y, w, h) as we would normally do
	# with OpenCV
	x1 = rect.left()
	y1 = rect.top()
	w1 = rect.right() - x1
	h1 = rect.bottom() - y1

	# return a tuple of (x, y, w, h)
	return (x1, y1, w1, h1)
 
# extract 68 coordinate from shape object 
def shape_to_np(shape, dtype = int):
  
  coords = np.zeros((68, 2), dtype=dtype)
  for i in range(0,68):
    coords[i] = (shape.part(i).x, shape.part(i).y)
  return coords # loop over the 68 facial landmarks and convert them

# calculate forehead distance to use in cropping image
def forehead_dist(coords):

  d = (np.sum(coords[42:47,1]) - np.sum(coords[36:41,1]))/ 6
  return d

# calculate angle using eye landmark points i.e 42 to 47 is right eye and 36 to 41 is left eye 
def required_angle(shape):
	
  val = (np.sum(shape[42:47,1]) - np.sum(shape[36:41,1]))/(np.sum(shape[42:47,0]) - np.sum(shape[36:41,0])) 
  angle = math.degrees(math.atan(val))
  return angle

#finally rotate image obtained by required_angle function
def rotate_image(image, shape):
  
  angle = required_angle(shape)
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  rotated_image = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return rotated_image

In [ ]:
def face_alignment(image):             # implementing face alignment
  image = np.array(image)
  image = image.astype(np.uint8)
  gray_image = image
  #gray_image = cv2.cvtColor(image ,cv2.COLOR_BGR2GRAY) # convert color image to grayscale image
  rects = detector(gray_image ,1)             # detect faces in the grayscale image
  if len(rects) > 0:

    images = []
    for (i, rect) in enumerate(rects):
      shape = predictor(image, rect)
      shape = shape_to_np(shape)

      rotated_image = rotate_image(image , shape)
      images.append(rotated_image)
    if len(rects) == 1 :
      return rotated_image
    else:
      return images
  else:
    #print("Error : number of detected face is zero, so we just return original image")
    return image